In [8]:
#| echo: False

import numpy as np
import fractions
import scipy
np.set_printoptions(formatter={'all':lambda x: str(fractions.Fraction(x).limit_denominator())})

## Eigenwertberechnung

### Hessenberg-Matrix

$$
H = \begin{pmatrix}
* & * & * & * & * \\
* & * & * & * & * \\
0 & * & * & * & * \\
0 & 0 & * & * & * \\
0 & 0 & 0 & * & *
\end{pmatrix}
$$

Eine Matrix $H \in \mathbb{R}^{N \times N}$ heißt (obere) Hessenberg-Matrix, wenn $H[n+2:N, n] = 0_{N-n-1}$ für $n = 1,...,N-2$ gilt.

::: {.callout-tip}

#### (4.3) Satz

Sei $A \in \mathbb{R}^{N \times N}$. Dann existiert eine **orthogonale** Matrix $Q \in \mathbb{R}^{N \times N}$, so dass 
$$
H = QAQ^{\top}
$$ 
eine *Hessenberg-Matrix* ist.

Die Berechnung von $Q$ benötigt $O(N^3)$ Operationen.

:::

![Berechnung der Hessenberg-Matrix](assets/hessenberg.jpeg)

In [11]:
A = np.array([
    [8, 12/5, -9/5],
    [12/5, 109/25, 12/25],
    [-9/5, 12/25, 116/25]
])

print(scipy.linalg.hessenberg(A))

[[8 -3 0]
 [-3 4 0]
 [0 0 5]]
